# Load the LLM and the bigvul's train dataframe

In [1]:
# run this for any section
import pandas as pd
from chat import PromptUtils

extract_code_content = PromptUtils.extract_code_content

# BIGVUL_TRAIN = "../bigvul-cleaned/clean_train.csv"
# dftr = pd.read_csv(BIGVUL_TRAIN)

dftr = pd.read_json("./container_data/primevul_train_cleaned_paired_full.jsonl", lines=True)

from chat import ChatGPTPrompter as gpt
from chat import CodeQwenPrompter as code_qwen
from chat import PromptUtils as pu
from utils import get_chatgpt_token as get_token

ch = gpt(get_token()) # Select ChatGPT 3.5 Turbo or 4o-mini

# ch = code_qwen(lock=None) # Selects CodeQwen1.5--7B-Chat

# FOR ChatGPT, create keys.yml file and add the following:
# openai:
#   chatgpt: "YOUR_TOKEN"

/home/shayan/.conda/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model Selected: gpt-4o-mini


## Introduciton of RAG

In [2]:
from rag import read_naive_code2code_clustered_rag_primevul_devigncleans, read_naive_full_header_rag, read_naive_full_bigvul_header_rag, read_naive_full_func_rag, read_naive_code2code_rag, read_naive_code2code_ext_rag, read_random, read_naive_code2code_clustered_rag, read_random_fair, read_naive_code2code_clustered_rag_primevul

# search_results = read_random(pv=True) # for mutation RQ1
# search_results = read_naive_code2code_clustered_rag_primevul(fo=True) # For Injection and Extension Methods - RQ1 and RQ3
search_results = read_naive_code2code_clustered_rag_primevul_devigncleans()
# search_results = read_naive_code2code_rag() # for RQ2's W\O Clustering
# search_results = read_random_fair() # for RQ2's W\O Retriever

In [3]:
# search_results[0].max_score, search_results[1].max_score, search_results[2].max_score, search_results[10].max_score, search_results[100].max_score 
len(search_results),search_results[0]

(11858,
 SearchItem(searchItemId='FFmpeg_463705bd1c644bbdded7bcf9f619bcb4203d562f_0_udp.c_nonvul.c_0.c', scoredCodeSnippets=[
   ScoredCodeSnippet(id='1758', body='{
 	int fput_needed, err, datagrams;
 	struct socket *sock;
 	s...', header='int __sys_recvmmsg(int fd, struct mmsghdr __user* mmsg, unsigned int vlen, unsigned int flags, struct timespec* timeout)', cluster='0', score=10.005205)
   ScoredCodeSnippet(id='274', body='{
     file_offset this_line = 0;
     file_offset first_comma...', header='intuit_diff_type (bool need_header, mode_t* p_file_type)', cluster='1', score=8.142351)
   ScoredCodeSnippet(id='3654', body='{
     GetBitContext gb;
     AC3HeaderInfo *hdr;
     int err;...', header='int avpriv_ac3_parse_header(AC3HeaderInfo* *phdr, const uint8_t* buf, size_t size)', cluster='2', score=7.387578)
   ScoredCodeSnippet(id='195611', body='{
   if (length == 0) {
     return 0;
   }
   assertx(m_bzFile)...', header='int64_t BZ2File::readImpl(char*  buf, int64_t length)', cl

In [4]:
from rag import flatten_search_items
from operator import attrgetter

def sort(search_results):
    return sorted(search_results, key=attrgetter('max_score'), reverse=True)

print(len(search_results)) 
search_results = flatten_search_items(search_results) # flattens search results in case the format is (1 clean) -> 5 vuls (for RQ2) instead of 1-1 matching, otherwise it doesn't change anything
search_results = sort(search_results) # sort the results (if clustering is used, this will be invalidated and the clustered sampling algorithm's sorting method will be used)
print(len(search_results))

# avg = 0
# max_avg = 0
# avg_m = 0
# for r in search_results:
#     if avg_m < r.avg_score:
#         avg_m = r.avg_score
#     avg += r.avg_score / len(search_results)
#     max_avg += r.max_score/len(search_results)

# print(avg, max_avg, avg_m)

11858
59289


In [5]:
def clustered_sampling(search_results):
    if search_results[0].scoredCodeSnippets[0].cluster == -1:
        print("Search results are for a non-clustered strategy, skipping clustred_sampling")
        return search_results

    # bigvul_cluster_order = [4,3,1,0,2] # sorted based on size
    # primevul - flaw only = [0, 4, 1, 2, 3]
    # primevul - full = [3, 2, 4, 0, 1]
    cluster_order = [0, 4, 1, 2, 3] # 
    cluster_indices = [0,0,0,0,0]

    new_search_results = []

    search_results_per_cluster = {0:[], 1:[], 2:[],3:[],4:[]}

    for sr in search_results:
        search_results_per_cluster[sr.scoredCodeSnippets[0].cluster].append(sr) # putting items of each cluster to it's array

    for i in range(5):
        search_results_per_cluster[i] = sort(search_results_per_cluster[i]) # sort each cluster's results

    for i in range(len(search_results)):
        cluster_number = cluster_order[i%5] # sampling from sorted clusters while starting from the largest cluster
        cluster_index = cluster_indices[cluster_number]

        if cluster_index < len(search_results_per_cluster[cluster_number]):
            new_search_results.append(search_results_per_cluster[cluster_number][cluster_index])
            cluster_indices[cluster_number] += 1
    
    print("Clustered Sampling was used...")
    return new_search_results

search_results = clustered_sampling(search_results)

Clustered Sampling was used...


In [6]:
search_results[0].max_score, search_results[1].max_score, search_results[2].max_score, search_results[10].max_score, search_results[100].max_score 

(32.65587, 83.97277, 54.31131, 30.80361, 25.200825)

In [7]:
### EXTRA Feb 2025
print(search_results[200])
get_clean_primevul(search_results[200]) # starting index = 16847

SearchItem(searchItemId='159850', scoredCodeSnippets=[
  ScoredCodeSnippet(id='180821', body='{
struct shash_instance *inst;
struct crypto_alg *alg;
struc...', header='static int hmac_create(struct crypto_template* tmpl, struct rtattr* *tb)', cluster='4', score=82.16385)
])


NameError: name 'get_clean_primevul' is not defined

In [10]:
# resuming_data_frame = pd.read_json("./generated/"somesetting".jsonl", orient="records", lines=True) # just in case more was needed, but didn't want to start from 0 again

In [8]:
import numpy as np
from rag import SearchItem
from chat import ConcurrencyUtils as cu
import json
def get_index(search_item: SearchItem):
    return str(search_item.best)


def get_vul(search_item: SearchItem):
    best_target_index = int(search_item.best)
    vul_record = dftr[dftr['index'] == best_target_index]
    return str(vul_record['processed_func'].values[0])

def get_fixed_vul(search_item: SearchItem):
    best_target_index = int(search_item.best)
    vul_record = dftr[dftr['index'] == best_target_index]
    return str(vul_record['func_after'].values[0])


def get_vul_lines(search_item: SearchItem):
    best_target_index = int(search_item.best)
    vul_record = dftr[dftr['index'] == best_target_index]
    flaw_lines = str(vul_record['flaw_line'].values[0]).strip()
    if len(flaw_lines) < 5:
        raise Exception(f"Invalid Vul_lines: {flaw_lines}")
    return flaw_lines


# def get_clean_bigvul(search_item: SearchItem):
#     best_target_index = int(search_item.best)
#     return str(dftr[dftr['index'] == best_target_index]['processed_func'].values[0])

# def get_clean_ext(search_item: SearchItem):
#     retrieved = None
#     try:
#         retrieved = get_clean_devign(search_item)
#     except Exception as e:
#         # print(f">>getting clean item, devign wasn't found {e}")
#         retrieved = get_clean_bigvul(search_item)
#         # print("<<retrieved from bigvul")
#         assert retrieved != None and len(retrieved) > 0
#     finally:
#         return retrieved

def get_clean_devign(search_item: SearchItem):
    with open(f"./../devign_code/{search_item.searchItemId}", 'r') as file:
        clean_item = file.read()
    return clean_item

def get_clean_primevul(search_item: SearchItem):
    # with open("container_data/primevul_train_cleaned_paired.jsonl", "r") as file:
    with open("container_data/primevul_train_cleaned_complete.jsonl", "r") as file:
        for line in file:
            record = json.loads(line)
            # print(record.get("index"), type(record.get("index")))
            if record.get("index") == int(search_item.searchItemId) and record.get("target") == 0:
                return record["processed_func"]
    
    return None

# Formulator Functions that take a search result and populates a prompt template

def get_injection_strategy_devign(search_item: SearchItem): # the Injection strategy while the clean's come from devign
    clean_item = get_clean_devign(search_item)
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_full_prompt(vul_code, clean_item, flaw_lines)


def get_extension_strategy_devign(search_item: SearchItem):
    clean_item = get_clean_devign(search_item)
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_ext_wl_prompt(clean_item, vul_code, flaw_lines)

def get_injection_strategy_primevul(search_item: SearchItem): # the Injection strategy while the clean's come from devign
    clean_item = get_clean_primevul(search_item)
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_full_prompt(vul_code, clean_item, flaw_lines)

def get_injection_strategy_primevul_devign_cleans(search_item: SearchItem): # the Injection strategy while the clean's come from devign
    clean_item = get_clean_devign(search_item)
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_full_prompt(vul_code, clean_item, flaw_lines)

def get_extension_strategy_primevul(search_item: SearchItem):
    clean_item = get_clean_primevul(search_item)
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_ext_wl_prompt(clean_item, vul_code, flaw_lines)

def get_extension_strategy_primevul_devign_cleans(search_item: SearchItem):
    clean_item = get_clean_devign(search_item)
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_ext_wl_prompt(clean_item, vul_code, flaw_lines)

def get_mutation_strategy(search_item: SearchItem):
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return pu.get_mutation_prompt(vul_code, flaw_lines)


# Generator Phase (can send concurrent requests)
indices, vuls, cleans, vul_lines, generated = cu.concurrent_prompter(
    prompt_creator_from_searchItem=get_extension_strategy_primevul_devign_cleans,
    index_retreiver=get_index,
    vul_retriever=get_vul,
    clean_retriever=get_clean_devign,
    vul_lines_retriever=get_vul_lines,
    search_results=search_results,
    llm=ch, target=6200, max_workers=8, is_all=False, resume_from_dataframe=None)  # for code Qwen set workers to 1 as there's only one model

df_result = pd.DataFrame({'vul': vuls, 'vul_lines': vul_lines, 'clean': cleans, 'generated': generated, 'idx':indices})
df_result.to_json(f"./generated/6k-primevul-extension-clustered_final_devigncleans.jsonl", orient="records", lines=True)


starting with index: 0
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Succe

In [18]:
# save the results of the Generator
df_result = pd.DataFrame({'vul': vuls, 'vul_lines': vul_lines, 'clean': cleans, 'generated': generated, 'idx':indices})
df_result.to_json(f"./generated/6k-primevul-injection-clustered_final_complete.jsonl", orient="records", lines=True)

In [20]:


indices2, vuls2, cleans2, vul_lines2, generated2 = cu.concurrent_prompter(
    prompt_creator_from_searchItem=get_extension_strategy_primevul,
    index_retreiver=get_index,
    vul_retriever=get_vul,
    clean_retriever=get_clean_primevul,
    vul_lines_retriever=get_vul_lines,
    search_results=search_results,
    llm=ch, target=6200, max_workers=8, is_all=False, resume_from_dataframe=None)

df_result = pd.DataFrame({'vul': vuls2, 'vul_lines': vul_lines2, 'clean': cleans2, 'generated': generated2, 'idx':indices2})
df_result.to_json(f"./generated/6k-primevul-extension_final_complete.jsonl", orient="records", lines=True)

starting with index: 0
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Succe

In [ ]:
# unused strategies

# def get_injection_bigvul(search_item: SearchItem):
#     clean_item = get_clean_bigvul(search_item)
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     return pu.get_full_prompt(vul_code, clean_item, flaw_lines)

# def get_injection_ext(search_item: SearchItem):
#     clean_item = get_clean_ext(search_item)
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     return pu.get_full_prompt(vul_code, clean_item, flaw_lines)
# failed strategies (or the ones that didn't do any better)
# def get_prompt4_devign(search_item: SearchItem):
#     clean_item = get_clean_devign(search_item)
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     return pu.get_full_prompt_without_examples_and_vardef_guide(vul_code, clean_item, flaw_lines)

# def get_prompt3_devign(search_item: SearchItem):
#     clean_item = get_clean_devign(search_item)
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     fixed = get_fixed_vul(search_item)
#     return pu.get_full_prompt_with_examples_and_vardef_guide(fixed, vul_code, clean_item, flaw_lines)

# def get_prompt5_devign(search_item: SearchItem):
#     clean_item = get_clean_devign(search_item)
#     vul_code = get_vul(search_item)
#     return pu.get_ext_prompt(clean_item, vul_code)
